In [1]:
# vscode에서 jupyter notebook 사용하려고 삽질할때 참고한 자료들:
# [Anaconda 가상환경 개념이해] https://yganalyst.github.io/pythonic/anaconda_env_1/
# [관리자권한 실행으로 해결] https://3rdscholar.tistory.com/81
from tqdm import tqdm
from glob import glob
from ast import literal_eval
import pandas as pd
import json
import csv
import os

In [2]:
# "라벨링데이터" 하위에 있는 모든 json파일 읽어오기 https://blr.design/blog/python-multiple-json-to-csv/
base = "../../"
files = glob(os.path.join(base, "k-fashion-dataset", "Training", "labels", "*", "*"), recursive=True)
# Validation 해당 데이터들도 마찬가지로 처리해주기
# files = glob(os.path.join(base, "k-fashion-dataset", "Validation", "labels", "*", "*"), recursive=True)

In [4]:
len(files)

In [8]:
# 읽어온 각각의 json을 필요한 정보만 뽑아서 row 하나로 만들기
data = []
for single_file in tqdm(files[:3]):
    with open(single_file, 'r', encoding='utf-8') as f:
        # Use 'try-except' to skip files that may be missing data
        try:
            json_file = json.load(f)
            file_id = json_file["데이터셋 정보"]["파일 번호"]
            file_name = json_file["데이터셋 정보"]["파일 이름"]
            top = json_file["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"]["상의"][0]
            data.append([file_id, file_name, top])
        except KeyError:
            print(f'Skipping {single_file}')
# Add header
data.insert(0, ['ID', 'File Name', 'Top'])
print("ALL DONE!")

In [9]:
len(data)

In [7]:
# csv로 변환
csv_filename = 'train2.csv'
with open(csv_filename, "w", encoding='utf-8', newline="") as f:
    writer = csv.writer(f)
    writer.writerows(data)

print("Updated CSV")

Updated CSV


In [8]:
# csv칼럼 내부 json 객체 flatten하기
# https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json
train = pd.read_csv('train2.csv',
                    converters = {'Top': literal_eval},
                    encoding='utf-8') # reading the csv file

In [9]:
train

,ID,File Name,Top
0,703911,(116)IMG_1.jpg,"{'색상': '네이비', '서브색상': '베이지', '카테고리': '블라우스', '..."
1,703912,(117)IMG_1.jpg,"{'색상': '네이비', '서브색상': '베이지', '카테고리': '블라우스', '..."
2,703913,(118)IMG_1.jpg,"{'색상': '네이비', '서브색상': '베이지', '카테고리': '블라우스', '..."
3,703914,(119)IMG_1.jpg,"{'색상': '네이비', '서브색상': '베이지', '카테고리': '블라우스', '..."
4,703917,(122)IMG_1.jpg,"{'색상': '네이비', '서브색상': '베이지', '카테고리': '블라우스', '..."
...,...,...,...
29995,1267305,20201117_001352_나염주름바지-158806142336533100_8018...,{}
29996,1267306,20201117_001352_나염주름바지-158806142379606500_5264...,{}
29997,1267316,20201117_001420_마버튼PT (6color)-158806137332809...,"{'색상': '오렌지', '서브색상': '화이트', '카테고리': '티셔츠', '소..."
29998,1267319,20201117_001420_마버튼PT (6color)-158806137689872...,{}


In [10]:
TOP = pd.json_normalize(train["Top"])
flattened = train[['ID', 'File Name']].join(TOP)
flattened

In [11]:
# 우리 모델에 필요한 칼럼만 남기기
dropped = flattened[['ID', 'File Name', '카테고리','소매기장']]
dropped

In [13]:
# one-hot 인코딩 하기
pd.set_option('display.max_columns', None)
targets = dropped.columns[2:]
final = pd.get_dummies(dropped, columns=targets, prefix_sep="_")

In [12]:
final

In [15]:
# ***주의***
final.to_csv(os.path.join(base, "machine-learning", "input", "preprocessed_train_ALL.csv"),
             index = False,
             encoding='utf-8-sig')
print("CSV Saved")

CSV Saved
